In [16]:
!wget --no-check -qP ~/Downloads/ http://d28rh4a8wq0iu5.cloudfront.net/ads1/code/bm_preproc.py
!wget --no-check -qP ~/Downloads/ http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fast

In [2]:
from bm_preproc import BoyerMoore

Implement versions of the naive exact matching and Boyer-Moore algorithms that additionally:

* count and return the number of character comparisons performed
* count and return the number of alignments tried

Roughly speaking, these measure how much work the two different algorithms are doing.

In [3]:
def naive(p, t):
    occurrences = []
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

def naive_with_counts(p, t):
    occurrences = []
    n_alignments = 0
    n_character_comparisons = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        n_alignments += 1
        match = True
        for j in range(len(p)):  # loop over characters
            n_character_comparisons += 1
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
        if match:
            occurrences.append(i)  # all chars matched; record
    print(f'''
occurrences: {occurrences}
alignments : {n_alignments}
comparisons: {n_character_comparisons}
''')
    return occurrences, n_alignments, n_character_comparisons


In [4]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

def boyer_moore_with_counts(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    n_alignments = 0
    n_character_comparisons = 0
    while i < len(t) - len(p) + 1:
        n_alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            n_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    print(f'''
occurrences: {occurrences}
alignments : {n_alignments}
comparisons: {n_character_comparisons}
''')
    return occurrences, n_alignments, n_character_comparisons

## Naive Example 1

In [5]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)


occurrences: [40]
alignments : 41
comparisons: 46

[40] 41 46


## Naive Example 2

In [6]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)


occurrences: [0, 19]
alignments : 20
comparisons: 35

[0, 19] 20 35


## Boyer-Moore Example 1

In [7]:
p = 'word'
t = 'there would have been a time for such a word'
lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)


occurrences: [40]
alignments : 12
comparisons: 15

[40] 12 15


## Boyer-Moore Example 2

In [8]:
p = 'needle'
t = 'needle need noodle needle'
p_bm = BoyerMoore(p, lowercase_alphabet)
occurrences, num_alignments, num_character_comparisons = boyer_moore_with_counts(p, p_bm, t)
print(occurrences, num_alignments, num_character_comparisons)


occurrences: [0, 19]
alignments : 5
comparisons: 18

[0, 19] 5 18


## Q1 & Q2

In [25]:
import os
from naive import readGenome

t = readGenome(os.path.expanduser('~') + '/Downloads/chr1.GRCh38.excerpt.fasta')
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [19]:
_, _, _ = naive_with_counts(p, t)


occurrences: [56922]
alignments : 799954
comparisons: 984143



## Q3

In [20]:
p_bm = BoyerMoore(p, 'ATCG')
_, _, _ = boyer_moore_with_counts(p, p_bm, t)


occurrences: [56922]
alignments : 127974
comparisons: 165191



## Q4

In [23]:
!wget --no-check -q https://d28rh4a8wq0iu5.cloudfront.net/ads1/code/kmer_index.py

In [67]:
from kmer_index import Index

# verification functions
def query_index_exact(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets

def query_index_approx(p, t, index, max_n_substitutions=2):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:
            offsets.append(i)
    return offsets

In [68]:
# check for errors
t = 'GCTACGATCTAGAATCTA'
p = 'TCTA'
index = Index(t, k=2)
print(query_index(p, t, index))

[7, 14]


In [70]:
t = readGenome(os.path.expanduser('~') + '/Downloads/chr1.GRCh38.excerpt.fasta')

In [72]:
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
index = Index(t, k=8)
hits = query_index(p, t, index)
print(hits)

[56922, 262042, 364263, 657496, 717706]


In [75]:
[t[hit:hit+24] == p for hit in hits]

[True, True, True, True, True]